In [7]:
#Import modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
#Enable inline plots
%matplotlib inline

In [9]:
#Import the streamflow data CSV and format columns
df=pd.read_csv('GageData.csv',dtype={'site_no':'str'},parse_dates=['datetime'])

In [10]:
#Add year and month columns
df['year'] = df['datetime'].map(lambda x: x.year)
df['month'] = df['datetime'].map(lambda x: x.month)

In [12]:
#Compute flow in cms
df['MeanFlow_cms'] = df['MeanFlow_cfs'] * 0.028316847 

In [13]:
#Set the index to full date
df.index = df.datetime

In [17]:
#Create data slices for pre- and post-Falls Lake
dfPre = df[:'1980-01-01']
dfPost = df['1983-12-31':]

### Compute min monthly flow

In [15]:
#Group data on month
byMonth = df.groupby('year')
dfMaxAnnual = byMonth['MeanFlow_cms'].max()
dfMaxAnnual.head()

year
1930     75.322813
1931    300.158578
1932    264.762519
1933    212.376352
1934    461.564606
Name: MeanFlow_cms, dtype: float64